## Imports

In [1]:
import time
import json
import numpy as np
import pandas as pd
from pymatgen.core.periodic_table import Species
from pymatgen.core.structure import Structure

## Read in JSON for Testing

In [2]:
with open('Materials_Project_NaNbO3.json') as json_file:
    nanbo3_dct = json.load(json_file)
for key in list(nanbo3_dct.keys()):
    nanbo3_dct[key]['structures'] = [Structure.from_dict(s) for s in nanbo3_dct[key]['structures']]

# Benchmark gii_calculator.py

## Compute Matminer GII - Benchmarking

In [3]:
from matminer.featurizers.structure.bonding import GlobalInstabilityIndex
# matminer GlobalInstabilityIndex uses bvparm16.cif, which is the default for GIICalculator

In [4]:
benchmark_structure = nanbo3_dct['NaNbO3']['structures'][1]

In [5]:
start_time = time.time()
mm_gii_calc = GlobalInstabilityIndex(r_cut=4)
mm_gii = mm_gii_calc.featurize(benchmark_structure)[0]
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.45648813247680664 seconds ---


In [6]:
mm_gii

0.22741774664473707

## Compute GIICalculator GII

In [7]:
from gii_calculator import GIICalculator

In [8]:
start_time = time.time()
#gii_calc = GIICalculator(method='CrystalNN')
gii_calc = GIICalculator(method='Cutoff', cutoff=4)
gii = gii_calc.GII(benchmark_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.10227823257446289 seconds ---


In [9]:
gii

0.2274177466447371

In [10]:
gii_calc.params_dict

{'Cation': [Species Na+, Species Nb5+],
 'Anion': [Species O2-, Species O2-],
 'R0': [1.803, 1.911],
 'B': [0.37, 0.37]}

# Benchmark site_optimization.py

## Perform the Site Optimization

In [12]:
from site_optimization import SiteClusterOptimization

In [18]:
sco = SiteClusterOptimization()

In [19]:
start_time = time.time()
final_structure = sco.minimize_cluster_di_squared(benchmark_structure, site_ind=16) # pass the index of site to be optimized
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 3.984825849533081 seconds ---


In [20]:
start_time = time.time()
start_gii = gii_calc.GII(benchmark_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.14157724380493164 seconds ---


In [21]:
start_gii

0.22741774664473707

In [22]:
start_time = time.time()
final_gii = gii_calc.GII(final_structure, use_sym=False)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.3294098377227783 seconds ---


In [23]:
final_gii

0.2222068229123708

## Perform the Structure Optimization 

In [12]:
from site_optimization import GIIMinimizer

In [25]:
gii_minimizer = GIIMinimizer(benchmark_structure, convergence_tolerance=0.01)
# All sites optimized until optimization of said site changes GII by < convergence_tolerance

In [26]:
optimized_structure = gii_minimizer.gii_minimization(opt_method='max')

Step 1 complete; 0.22741774664473727 --> 0.21685468242809414
Step 2 complete; 0.21685468242809414 --> 0.21253671761744256
Step 3 complete; 0.21253671761744256 --> 0.20319583095480914
Step 4 complete; 0.20319583095480914 --> 0.19929305787714166
Step 5 complete; 0.19929305787714166 --> 0.17406125117144214
Step 6 complete; 0.17406125117144214 --> 0.1541425987472521
Step 7 complete; 0.1541425987472521 --> 0.14735253314386446
Step 8 complete; 0.14735253314386446 --> 0.12605574185146776
Step 9 complete; 0.12605574185146776 --> 0.1172540398089425
Step 10 complete; 0.1172540398089425 --> 0.11532954906197346
Step 11 complete; 0.11532954906197346 --> 0.11098630269185894
Step 12 complete; 0.11098630269185894 --> 0.10514996486333666
Step 13 complete; 0.10514996486333666 --> 0.10514996486333666
Step 14 complete; 0.10514996486333666 --> 0.10514996486333666
Step 15 complete; 0.10514996486333666 --> 0.10512042010554813
Step 16 complete; 0.10512042010554813 --> 0.1051185729262532
Step 17 complete; 0.10

In [27]:
# Get the change in GII following optimization for each site in the structure, show that it is < convergence_tolerance
for i in range(len(gii_minimizer.diffs)):
    print('Specie: %s, delGII of optimization step: %s' % (optimized_structure[i].specie, gii_minimizer.diffs[i]))

Specie: Na+, delGII of optimization step: 0.006511574041070112
Specie: Na+, delGII of optimization step: 0.007380391101802525
Specie: Na+, delGII of optimization step: 0.0018892742339008256
Specie: Na+, delGII of optimization step: 0.00614137470116205
Specie: Nb5+, delGII of optimization step: 2.9544757788532716e-05
Specie: Nb5+, delGII of optimization step: 0.0
Specie: Nb5+, delGII of optimization step: 1.847179294925394e-06
Specie: Nb5+, delGII of optimization step: 0.0
Specie: O2-, delGII of optimization step: 0.008801702042525256
Specie: O2-, delGII of optimization step: 0.001924490746969043
Specie: O2-, delGII of optimization step: 0.004343246370114517
Specie: O2-, delGII of optimization step: 0.0058363378285222756
Specie: O2-, delGII of optimization step: 0.006790065603387657
Specie: O2-, delGII of optimization step: 2.7755575615628914e-17
Specie: O2-, delGII of optimization step: 0.002665507676445314
Specie: O2-, delGII of optimization step: 0.0010834205073378123
Specie: O2-, de

# Benchmark parameterization.py

In [11]:
from parameterization import BVParamOptimization
from parameterization import GeneralBVParamOptimizationOuterLoop
from parameterization import CompositionSpecificBVParamOptimizationOuterLoop
from pymatgen.core.periodic_table import Specie

In [12]:
### Only consider structures with 20 atoms or fewer to speed up parameterization test
cmpd = 'NaNbO3'
ss = nanbo3_dct[cmpd]['structures']
short_inds = [i for i in range(len(ss)) if len(ss[i]) <= 20]

short_structures = [ss[i] for i in short_inds]
short_energies = [nanbo3_dct['NaNbO3']['energies'][i] for i in short_inds]
short_dct = {cmpd: {'structures': short_structures, 'energies': short_energies}}

cation = Specie('Nb', 5)
anion = Specie('O', -2)
cations_anions = [(Specie('Nb', 5), Specie('O', -2)), (Specie('Na', 1), Specie('O', -2))]
options = {'gtol': 0.01, 'xtol': 0.01, 'barrier_tol': 0.01, 'disp': True, 'verbose': 0} # Changed from default

## Parameterization that minimizes site discrepancy factor RMSD

### Composition-Specific Parameterization - R0 Only

In [21]:
rmsd_cs_parameterization = CompositionSpecificBVParamOptimizationOuterLoop(short_dct, [cmpd], gii_calc.params_dict)

In [24]:
rmsd_cs_parameterization.parameter_optimization(obj_func='gii_gs', parameterize='both')

[1.803, 1.911, 0.37, 0.37]
-0.31195916177186295
-0.3119593502018366
-0.3119589033101902
-0.3119594806739683
-0.31195913918541185
0.8212405346231583
0.8212405261822375
0.821240547078119
0.821240531587492
0.8212405272654437
0.8631381992945217
0.8631381589079326
0.8631381817516841
0.8631381472926805
0.8631382050875429
0.40813352456467644
0.40813352813888426
0.40813344984288635
0.4081334260118472
0.4081334549157807
-0.874008557277266
-0.8740086095741222
-0.8740085970322924
-0.8740086169572951
-0.8740085829897546
-0.2692865789989449
-0.269287032919748
-0.2692859160716786
-0.2692872104927309
-0.26928651173700974
0.4245028214250016
0.42450258472588964
0.4245031076538636
0.4245026177671756
0.4245028254108253
0.67176003686702
0.6717597045194879
0.6717602384728447
0.6717596415298505
0.6717600543019862
-0.09990089517286946
-0.09990145176888154
-0.09989985821678538
-0.09990163953683207
-0.09990080107374408
0.6016380189271193
0.6016372125984608
0.6016393615319608
0.6016370882510353
0.60163813165039

In [25]:
rmsd_cs_parameterization.updated_params_by_composition, rmsd_cs_parameterization.starting_params

({'NaNbO3': {'Cation': [Species Na+, Species Nb5+],
   'Anion': [Species O2-, Species O2-],
   'R0': [1.698, 1.961],
   'B': [0.419, 0.347]}},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]})

### Single Parameter Optimization

In [21]:
short_structures

[Structure Summary
 Lattice
     abc : 5.581053514769859 5.581053514769859 7.944256
  angles : 90.0 90.0 90.60105371470894
  volume : 247.43532826035178
       A : 3.925647 -3.967046 0.0
       B : 3.925647 3.967046 0.0
       C : 0.0 0.0 7.944256
 PeriodicSite: Na+ (3.9256, 0.0864, 5.9582) [0.4891, 0.5109, 0.7500]
 PeriodicSite: Na+ (3.9256, -0.0864, 1.9861) [0.5109, 0.4891, 0.2500]
 PeriodicSite: Na+ (3.9256, -3.9393, 5.9582) [0.9965, 0.0035, 0.7500]
 PeriodicSite: Na+ (3.9256, 3.9393, 1.9861) [0.0035, 0.9965, 0.2500]
 PeriodicSite: Nb5+ (1.9628, 1.9835, 0.0000) [0.0000, 0.5000, 0.0000]
 PeriodicSite: Nb5+ (1.9628, 1.9835, 3.9721) [0.0000, 0.5000, 0.5000]
 PeriodicSite: Nb5+ (1.9628, -1.9835, 0.0000) [0.5000, 0.0000, 0.0000]
 PeriodicSite: Nb5+ (1.9628, -1.9835, 3.9721) [0.5000, 0.0000, 0.5000]
 PeriodicSite: O2- (5.5732, -1.9318, 1.9861) [0.9533, 0.4664, 0.2500]
 PeriodicSite: O2- (2.2781, 1.9318, 5.9582) [0.0467, 0.5336, 0.7500]
 PeriodicSite: O2- (5.5732, 1.9318, 5.9582) [0.4664, 

In [23]:
single_parameterization_rmsd = BVParamOptimization([short_structures], [short_energies], 
                                                          gii_calc.params_dict, [cation], [anion], 
                                                          obj_func='di2_rmsd', options=options)

In [24]:
rmsd_optimized_param_dict = single_parameterization_rmsd.param_optimizer()

0.43217634981643743
0.43217600192001077
63.21939147166733
63.219399469371446
0.4140825213410144
0.4140829458064924
0.6421104554175449
0.6421101245876905
0.2074310114107347
0.20743065705585592
0.3224547405068849
0.3224551528855676
0.07429075164423778
0.07429094301606244
0.20907175831980218
0.20907140381564515
0.08023512139559828
0.08023489434192736
`xtol` termination condition is satisfied.
Number of iterations: 9, function evaluations: 18, CG iterations: 8, optimality: 6.60e+00, constraint violation: 0.00e+00, execution time: 1e+01 s.


In [25]:
rmsd_optimized_param_dict, gii_calc.params_dict

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.947],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]})

### Multiple Parameter Optimization

In [26]:
rmsd_pol = GeneralBVParamOptimizationOuterLoop(short_dct, cations_anions, gii_calc.params_dict)
rmsd_pol.parameter_optimization(obj_func='di2_rmsd', init_steps=1)

(Species Nb5+, Species O2-) 1
1.911 0.37
0.43217634981643743
0.43217600192001077
63.21939147166733
63.219399469371446
0.4140825213410144
0.4140829458064924
0.6421104554175449
0.6421101245876905
0.2074310114107347
0.20743065705585592
0.3224547405068849
0.3224551528855676
0.07429075164423778
0.07429094301606244
0.20907175831980218
0.20907140381564515
0.08023512139559828
0.08023489434192736
`xtol` termination condition is satisfied.
Number of iterations: 9, function evaluations: 18, CG iterations: 8, optimality: 6.60e+00, constraint violation: 0.00e+00, execution time: 1e+01 s.
1.947 0.37
(Species Na+, Species O2-) 1
1.803 0.37
0.11400219061832283
0.11400217674652065
2.929394163023829
2.9293945290047945
0.11964710051488511
0.11964712771839521
0.11232720162549459
0.11232720889891686
0.11177575186306263
0.11177575169816566
0.11177546249795854
0.11177546249600917
`gtol` termination condition is satisfied.
Number of iterations: 6, function evaluations: 12, CG iterations: 5, optimality: 7.22e-

In [29]:
rmsd_pol.starting_params, rmsd_pol.updated_params

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.811, 1.947],
  'B': [0.37, 0.37]})

## Parameterization considering GII-DFT Energetics Relationship

### Composition-Specific Parameterization

In [31]:
gs_cs_parameterization = CompositionSpecificBVParamOptimizationOuterLoop(short_dct, [cmpd], gii_calc.params_dict)

In [32]:
gs_cs_parameterization.parameter_optimization(obj_func='gii_gs', parameterize='both')

[1.803, 1.911, 0.37, 0.37]
-0.31195916177186295
-0.3119593502018366
-0.3119589033101902
-0.3119594806739683
-0.31195913918541185
0.8212405346231583
0.8212405261822375
0.821240547078119
0.821240531587492
0.8212405272654437
0.8631381992945217
0.8631381589079326
0.8631381817516841
0.8631381472926805
0.8631382050875429
0.40813352456467644
0.40813352813888426
0.40813344984288635
0.4081334260118472
0.4081334549157807
-0.874008557277266
-0.8740086095741222
-0.8740085970322924
-0.8740086169572951
-0.8740085829897546
-0.2692865789989449
-0.269287032919748
-0.2692859160716786
-0.2692872104927309
-0.26928651173700974
0.4245028214250016
0.42450258472588964
0.4245031076538636
0.4245026177671756
0.4245028254108253
0.67176003686702
0.6717597045194879
0.6717602384728447
0.6717596415298505
0.6717600543019862
-0.09990089517286946
-0.09990145176888154
-0.09989985821678538
-0.09990163953683207
-0.09990080107374408
0.6016380189271193
0.6016372125984608
0.6016393615319608
0.6016370882510353
0.60163813165039

### Single Parameter Optimization

In [28]:
single_parameterization_gii = BVParamOptimization([short_structures], [short_energies], 
                                                         gii_calc.params_dict, [cation], [anion], options=options)

In [29]:
gii_optimized_param_dict = single_parameterization_gii.param_optimizer()

-0.31195916177186295
-0.3119589033101902
0.8266033357753323
0.826603340335776
0.8535480718045818
0.8535480312807145
-0.39097757570365277
-0.3909774480010062
0.8219541046914551
0.8219541054474953
0.8597353823300886
0.8597353358383644
-0.006679020775964051
-0.006678288212894901
0.8637759252091005
0.8637758750450728
0.8898998049832447
0.8898997369931558
0.712875293375542
0.7128769142693724
0.9013424941399776
0.9013424218891131
`gtol` termination condition is satisfied.
Number of iterations: 11, function evaluations: 22, CG iterations: 10, optimality: 4.62e-03, constraint violation: 0.00e+00, execution time: 3.1e+01 s.


In [30]:
gii_optimized_param_dict, gii_calc.params_dict

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.961],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]})

### Multiple Parameter Optimization

In [34]:
gii_pol = GeneralBVParamOptimizationOuterLoop(short_dct, cations_anions, gii_calc.params_dict)

In [35]:
gii_pol.parameter_optimization(init_steps=1, max_steps=3)

(Species Nb5+, Species O2-) 1
1.911 0.37
-0.31195916177186295
-0.3119589033101902
0.8266033357753323
0.826603340335776
0.8535480718045818
0.8535480312807145
-0.39097757570365277
-0.3909774480010062
0.8219541046914551
0.8219541054474953
0.8597353823300886
0.8597353358383644
-0.006679020775964051
-0.006678288212894901
0.8637759252091005
0.8637758750450728
0.8898998049832447
0.8898997369931558
0.712875293375542
0.7128769142693724
0.9013424941399776
0.9013424218891131
0.9075435230816551
0.9075434843020624
0.908019140737167
0.9080191581905526
0.9080651551777972
0.9080651648362212
0.9080760146495126
0.9080760214232637
`gtol` termination condition is satisfied.
Number of iterations: 17, function evaluations: 30, CG iterations: 14, optimality: 8.65e-04, constraint violation: 0.00e+00, execution time: 4.2e+01 s.
1.957 0.37
(Species Na+, Species O2-) 1
1.803 0.37
0.9080332128113098
0.9080331561312214
0.8647245005521961
0.8647245075482657
0.928128455842581
0.9281284732688118
0.9102386228649307
0.

In [36]:
gii_pol.starting_params, gii_pol.updated_params

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.784, 1.958],
  'B': [0.37, 0.37]})